In [ ]:
import pyrealsense2 as rs
import time

def get_color_sensor(profile):
    dev = profile.get_device()
    for s in dev.sensors:
        name = s.get_info(rs.camera_info.name).lower()
        if "color" in name or "rgb" in name:
            return s
    return None

def print_supported_options(sensor):
    print("Supported options and ranges:")
    for opt in [o for o in dir(rs.option) if o.isupper() == False]:
        pass
    opts_to_try = [
        rs.option.enable_auto_exposure, rs.option.exposure,
        rs.option.enable_auto_white_balance, rs.option.white_balance,
        rs.option.gain, rs.option.brightness, rs.option.contrast, rs.option.saturation
    ]
    for opt in opts_to_try:
        try:
            name = opt.name
            if sensor.supports(opt):
                mn = sensor.get_option_range(opt).min
                mx = sensor.get_option_range(opt).max
                step = sensor.get_option_range(opt).step
                val = sensor.get_option(opt)
                print(f"  {name}: supported, range=({mn}, {mx}, step={step}), current={val}")
            else:
                print(f"  {opt.name}: NOT supported")
        except Exception as e:
            print("  error checking", opt, e)

def stabilize_and_lock_exposure(profile, pipeline, stabilize_frames=30):

    color_sensor = get_color_sensor(profile)
    if color_sensor is None:
        raise RuntimeError("Color sensor not found")

    try:
        if color_sensor.supports(rs.option.enable_auto_exposure):
            color_sensor.set_option(rs.option.enable_auto_exposure, 1)
        if color_sensor.supports(rs.option.enable_auto_white_balance):
            color_sensor.set_option(rs.option.enable_auto_white_balance, 1)
    except Exception as e:
        print("Warning: could not enable auto options:", e)

    for _ in range(stabilize_frames):
        pipeline.wait_for_frames()
        time.sleep(0.02)

    vals = {}
    try:
        if color_sensor.supports(rs.option.exposure):
            vals['exposure'] = color_sensor.get_option(rs.option.exposure)
        if color_sensor.supports(rs.option.white_balance):
            vals['white_balance'] = color_sensor.get_option(rs.option.white_balance)
        if color_sensor.supports(rs.option.gain):
            vals['gain'] = color_sensor.get_option(rs.option.gain)
    except Exception as e:
        print("Warning: could not read some options:", e)

    print("Stabilized values:", vals)

    try:
        if color_sensor.supports(rs.option.enable_auto_exposure):
            color_sensor.set_option(rs.option.enable_auto_exposure, 0)
        if color_sensor.supports(rs.option.enable_auto_white_balance):
            color_sensor.set_option(rs.option.enable_auto_white_balance, 0)

        if 'exposure' in vals and color_sensor.supports(rs.option.exposure):
            color_sensor.set_option(rs.option.exposure, vals['exposure'])
        if 'white_balance' in vals and color_sensor.supports(rs.option.white_balance):
            color_sensor.set_option(rs.option.white_balance, vals['white_balance'])
        if 'gain' in vals and color_sensor.supports(rs.option.gain):
            color_sensor.set_option(rs.option.gain, vals['gain'])
    except Exception as e:
        print("Warning: could not disable autos or set manual values:", e)

    readback = {}
    try:
        if color_sensor.supports(rs.option.exposure):
            readback['exposure'] = color_sensor.get_option(rs.option.exposure)
        if color_sensor.supports(rs.option.white_balance):
            readback['white_balance'] = color_sensor.get_option(rs.option.white_balance)
        if color_sensor.supports(rs.option.gain):
            readback['gain'] = color_sensor.get_option(rs.option.gain)
    except Exception as e:
        print("Warning: unable to read back values:", e)

    print("Locked values (readback):", readback)
    return readback

pipeline = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
profile = pipeline.start(cfg)
try:
    sensor = get_color_sensor(profile)
    print_supported_options(sensor)

    final_vals = stabilize_and_lock_exposure(profile, pipeline, stabilize_frames=30)
finally:
    pipeline.stop()


Supported options and ranges:
  enable_auto_exposure: supported, range=(0.0, 1.0, step=1.0), current=0.0
  exposure: supported, range=(1.0, 10000.0, step=1.0), current=166.0
  enable_auto_white_balance: supported, range=(0.0, 1.0, step=1.0), current=0.0
  white_balance: supported, range=(2800.0, 6500.0, step=10.0), current=4600.0
  gain: supported, range=(0.0, 128.0, step=1.0), current=64.0
  brightness: supported, range=(-64.0, 64.0, step=1.0), current=0.0
  contrast: supported, range=(0.0, 100.0, step=1.0), current=50.0
  saturation: supported, range=(0.0, 100.0, step=1.0), current=64.0
Stabilized values (will lock): {'exposure': 166.0, 'white_balance': 4600.0, 'gain': 64.0}
Locked values (readback): {'exposure': 166.0, 'white_balance': 4600.0, 'gain': 64.0}


In [2]:
import json

preset = {'exposure': 166.0, 'white_balance': 4600.0, 'gain': 64.0}
with open('rs_locked_preset.json','w') as f:
    json.dump(preset, f, indent=2)
print("Saved preset -> rs_locked_preset.json")


Saved preset -> rs_locked_preset.json


In [3]:
import pyrealsense2 as rs
import json
import time

def get_color_sensor(profile):
    dev = profile.get_device()
    for s in dev.sensors:
        name = s.get_info(rs.camera_info.name).lower()
        if "color" in name or "rgb" in name:
            return s
    return None

def apply_preset(profile, preset):

    sensor = get_color_sensor(profile)
    if sensor is None:
        raise RuntimeError("Color sensor not found")

    try:
        if sensor.supports(rs.option.enable_auto_exposure):
            sensor.set_option(rs.option.enable_auto_exposure, 0)
    except Exception as e:
        print("could not disable auto exposure:", e)

    try:
        if sensor.supports(rs.option.enable_auto_white_balance):
            sensor.set_option(rs.option.enable_auto_white_balance, 0)
    except Exception as e:
        print("could not disable auto white balance:", e)

    try:
        if 'exposure' in preset and sensor.supports(rs.option.exposure):
            sensor.set_option(rs.option.exposure, float(preset['exposure']))
        if 'white_balance' in preset and sensor.supports(rs.option.white_balance):
            sensor.set_option(rs.option.white_balance, float(preset['white_balance']))
        if 'gain' in preset and sensor.supports(rs.option.gain):
            sensor.set_option(rs.option.gain, float(preset['gain']))
    except Exception as e:
        print("could not set some options:", e)

    readback = {}
    try:
        if sensor.supports(rs.option.exposure):
            readback['exposure'] = sensor.get_option(rs.option.exposure)
        if sensor.supports(rs.option.white_balance):
            readback['white_balance'] = sensor.get_option(rs.option.white_balance)
        if sensor.supports(rs.option.gain):
            readback['gain'] = sensor.get_option(rs.option.gain)
    except Exception as e:
        print("could not read back options:", e)
    print("Applied preset, readback:", readback)
    return readback

pipeline = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.color, 1280,720, rs.format.bgr8, 30)
profile = pipeline.start(cfg)
time.sleep(0.1)

with open('rs_locked_preset.json','r') as f:
    preset = json.load(f)

apply_preset(profile, preset)


Applied preset, readback: {'exposure': 166.0, 'white_balance': 4600.0, 'gain': 64.0}


{'exposure': 166.0, 'white_balance': 4600.0, 'gain': 64.0}